In [1]:
import pandas as pd

#INCAR data from bbsr
#https://www.inkar.de
#Vor allem: Anteil Fertiggestellte Wohnungen / Wohngebäude mit erneuerbarer Heizenergie, 2017-2019

#convert pandas series from German number format (1.000,5) to python number format (1000.5)
def make_column_numeric(se):
    se = se.str.replace('.','',regex=True)
    se = se.str.replace(',','.',regex=True)
    se = pd.to_numeric(se)
    return se

#read dataset
df = pd.read_csv("data/incar.csv", delimiter=';', header=[0,1])
df.columns = df.columns.map('_'.join)

#rename unnamed headers
df.rename({"Kennziffer_Unnamed: 0_level_1": "Kennziffer","Raumeinheit_Unnamed: 1_level_1":"Raumeinheit","Aggregat_Unnamed: 2_level_1":"Aggregated"}, axis=1,inplace=True)

#calculate neubauwohnungen total and neubauwohnungen renewable total
years = ["2018","2019"]
for year in years:
    #make columns numerics
    neubau_einwohner = "Neubauwohnungen je Einwohner_" + str(year)
    einwohner = 'Bevölkerung gesamt_' + str(year)
    neubauwohnungen_renewable_anteil = "Fertiggestellte Wohnungen mit erneuerbarer Heizenergie_" + str(year)
    neubauwohngebäude_renewable_anteil = "Fertiggestellte Wohngebäude mit erneuerbarer Heizenergie_" + str(year)
    
    df[neubau_einwohner] = make_column_numeric(df[neubau_einwohner]) 
    df[einwohner] = make_column_numeric(df[einwohner]) 
    df[neubauwohnungen_renewable_anteil] = make_column_numeric(df[neubauwohnungen_renewable_anteil]) / 100
    df[neubauwohngebäude_renewable_anteil] = make_column_numeric(df[neubauwohngebäude_renewable_anteil]) / 100

    #Neubauwohnungen = Neubauwohnungen je Einwohner * Einwohner 
    neubau = "neubauwohnungen_" + str(year)
    df[neubau] = df[neubau_einwohner] *  df[einwohner]

    #Neubauwohnungen renewable = Neubauwognungen * Anteil Renewalbe
    #Wohnungen vs. Wohngebäude: aus Dokumentation: "Im Gegensatz zum Indikator Fertiggestellte Wohngebäude mit erneuerbarer Heizenergie werden hier 
    #Errichtungen von Wohnungen in Mehrfamilienhäusern gegenüber Ein- und Zweifamilienhäuser stärker gewichtet." Bei Wohngebäuden wäre ein Mehrfamilienhaus
    #genausoviel wert wie ein Einfamilienhaus
    neubauwohnungen_renewable =  "neubauwohnungen_erneuerbareheiz_" + str(year)
    df[neubauwohnungen_renewable] = df[neubau] *  df[neubauwohnungen_renewable_anteil]

    neubauwohngebäude_renewable =  "neubauwohngebäude_erneuerbareheiz_" + str(year)
    df[neubauwohngebäude_renewable] = df[neubau] *  df[neubauwohngebäude_renewable_anteil]

df.to_csv("data/incar_output.csv")


In [81]:
#https://www-genesis.destatis.de/genesis/online#astructure
#Baufertigstellungen neuer Gebäude: Deutschland, Jahre, Gebäudeart, Energieverwendung, Energieart
#von 2014 - 2021, auf Deutschlandebene

import plotly.graph_objects as go
import numpy as np

df2_org = pd.read_csv("data/newbuildings_germany.csv",delimiter=";", encoding='latin1')

def plot_energy_for_building_type(building_type):
    
    df2 = df2_org.copy()

    #Nur den Typ des Gebäudes auswählen
    df2 = df2.loc[df2["2_Auspraegung_Label"] == building_type]

    #Nur Zum Heizen verwendete Primärenergie
    df2 = df2.loc[df2["3_Auspraegung_Code"] == "ENR-HZG-PRM-01"]

    df2.rename({"Zeit": "year","BAU005__Neue_Gebaeude__Anzahl":"buildings","4_Auspraegung_Label":"energy"}, axis=1,inplace=True)

    #make building number numeric
    df2["buildings"] = df2["buildings"].str.replace('-','0',regex=True)
    df2["buildings"] = df2["buildings"].str.replace('.','0',regex=True)
    df2["buildings"] = pd.to_numeric(df2["buildings"])


    #pivot table
    pivot = pd.pivot_table(df2, index='year',columns='energy', values='buildings',aggfunc=np.sum)
    pivot = pivot.reset_index(level=0)

    #create plot
    colors = px.colors.qualitative.Plotly
    fig = go.Figure()

    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Gas"], mode = 'lines', line=dict(color=colors[0]),name="Gas"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Öl"], mode = 'lines', line=dict(color=colors[1]),name="Öl"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Strom"], mode = 'lines', line=dict(color=colors[2]),name="Strom"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Fernwärme/Fernkälte"], mode = 'lines', line=dict(color=colors[3]),name="Fernwärme/Fernkälte"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Holz"], mode = 'lines', line=dict(color=colors[4]),name="Holz"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Umweltthermie"], mode = 'lines', line=dict(color=colors[5]),name="Umweltthermie"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Biogas/Biomethan"], mode = 'lines', line=dict(color=colors[6]),name="Biogas/Biomethan"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Geothermie"], mode = 'lines', line=dict(color=colors[7]),name="Geothermie"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Solarthermie"], mode = 'lines', line=dict(color=colors[8]),name="Solarthermie"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Sonstige Energie"], mode = 'lines', line=dict(color=colors[9]),name="Sonstige Energie"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Sonstige Biomasse"], mode = 'lines', line=dict(color=colors[0]),name="Sonstige Biomasse"))
    fig.add_traces(go.Scatter(x=pivot['year'], y = pivot["Keine Energie"], mode = 'lines', line=dict(color=colors[1]),name="Keine Energie"))

    fig.show()


In [82]:
##choose on of those labels
print(df2_org["2_Auspraegung_Label"].unique())

#and plot
plot_energy_for_building_type("Wohngebäude")

['Wohngebäude' 'Wohngebäude mit 1 Wohnung' 'Wohngebäude mit 2 Wohnungen'
 'Wohngebäude mit 3 oder mehr Wohnungen' 'Wohnheime'
 'Wohngebäude mit Eigentumswohnungen' 'Nichtwohngebäude' 'Anstaltsgebäude'
 'Büro- und Verwaltungsgebäude' 'Landwirtschaftliche Betriebsgebäude'
 'Nichtlandwirtschaftliche Betriebsgebäude' 'Fabrik- und Werkstattgebäude'
 'Handelsgebäude' 'Warenlagergebäude' 'Hotels und Gaststätten'
 'Sonstige Nichtwohngebäude' 'Ausgewählte Infrastrukturgebäude']
